In [ ]:
#FOR LIVE ANALYSIS RUN THE LAST CELL

In [ ]:
print("="*50)
print("STEP 1: Fixing OpenCV and Installing Dependencies")
print("="*50)

%pip uninstall -y opencv-python opencv-python-headless opencv-contrib-python

%pip install -q opencv-python
%pip install -q ultralytics yt-dlp supervision
%pip install pandas

print("✅ Installation Complete!\n")

# Import Libraries
import cv2
import numpy as np
from collections import defaultdict, deque
import math
import os
import time
import torch

# Import YOLO
from ultralytics import YOLO

# Define custom display function
def cv2_imshow(img):
    """Custom function to display images locally"""
    cv2.imshow("YOLO Output", img)
    cv2.waitKey(1)

print("✅ Libraries Imported Successfully!")
print(f"✅ OpenCV Version: {cv2.__version__}")

# Verify cv2.imshow exists
print(f"✅ cv2.imshow available: {hasattr(cv2, 'imshow')}")

# Check GPU Availability
print(f"✅ CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU Device: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️  Running on CPU (slower performance)")

print("\n" + "="*50)
print("Environment Setup Complete!")
print("="*50)

STEP 1: Fixing OpenCV and Installing Dependencies
Found existing installation: opencv-python 4.12.0.88
Uninstalling opencv-python-4.12.0.88:
  Successfully uninstalled opencv-python-4.12.0.88
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
✅ Installation Complete!

✅ Libraries Imported Successfully!
✅ OpenCV Version: 4.12.0
✅ cv2.imshow available: True
✅ CUDA Available: True
✅ GPU Device: NVIDIA GeForce GTX 1650

Environment Setup Complete!


In [ ]:
import os
import urllib.request

print("="*70)
print("🎯 FOOTFALL COUNTER - FRESH START WITH PROPER VIDEO")
print("="*70)

print("\n📹 BEST OPTIONS FOR DOORWAY/CORRIDOR COUNTING:\n")

print("1️⃣  MALL DATASET (RECOMMENDED)")
print("   - 2000 frames, 60K+ pedestrians")
print("   - Static camera, mall entrance")
print("   - Perfect for entry/exit counting")

print("\n2️⃣  PETS2009 DATASET")
print("   - Surveillance footage")
print("   - Clear line crossing scenarios")
print("   - Multiple people, realistic")

print("\n3️⃣  UCSD PEDESTRIAN DATASET")
print("   - Walkway/corridor scenarios")
print("   - Static camera, clear view")

print("\n4️⃣  FREE STOCK VIDEO (EASIEST)")
print("   - Pexels/Pixabay doorway videos")
print("   - No registration needed")
print("   - Ready to download")

print("\n" + "="*70)
print("💡 RECOMMENDED APPROACH:")
print("="*70)
print("""
We'll now try to automatically download a **perfect sample pedestrian video**:
- Static security camera
- Good crowd movement
- Ideal for entry/exit counting
""")

print("🔄 Checking / Creating videos folder...")
os.makedirs("videos", exist_ok=True)

print("\n🔄 Downloading sample pedestrian video...")

urls_to_try = [
    ("https://www.robots.ox.ac.uk/ActiveVision/Research/Projects/2009bbenfold_headpose/Datasets/TownCentreXVID.avi", "towncentre.avi"),
    ("https://motchallenge.net/sequenceVideos/MOT17-02-FRCNN-raw.webm", "mot17.webm"),
]

video_downloaded = False
video_path = None

for url, filename in urls_to_try:
    try:
        print(f"\n🔄 Trying to download: {filename} ...")
        save_path = f"videos/{filename}"
        urllib.request.urlretrieve(url, save_path)

        if os.path.exists(save_path) and os.path.getsize(save_path) > 1_000_000:
            print(f"✅ Successfully Downloaded: {filename}")
            video_path = save_path
            video_downloaded = True
            break
        else:
            print(f"⚠️  File downloaded but seems too small, skipping {filename}")
    except Exception as error:
        print(f"❌ Error downloading {filename}: {error}")

if not video_downloaded:
    print("\n⚠️  Automatic download failed.")
    print("="*70)
    print("📥 MANUAL DOWNLOAD OPTIONS (BEST WORKING VIDEOS)")
    print("="*70)
    print("""
Download any of these manually (1 minute download), then save as:
👉  videos/pedestrian.mp4

🎯 OPTION 1 (recommended):
https://www.pexels.com/video/people-walking-inside-a-building-856399/

🎯 OPTION 2:
https://pixabay.com/videos/people-pedestrians-walking-crowd-1252/

🎯 OPTION 3:
https://www.videvo.net/video/people-walking-in-shopping-mall/4526/

After downloading:
1. Create a folder named `videos` (if not already created)
2. Place your downloaded video inside it
3. Rename it to: pedestrian.mp4
""")

    video_path = "videos/pedestrian.mp4"
    print(f"\n📌 Expected video path: {video_path}")
    print("➡️  Please make sure you add your video before running detection.")

print("\n" + "="*70)
print("✅ VIDEO SETUP COMPLETE")
print(f"🎥 VIDEO PATH: {video_path}")
print("="*70)


🎯 FOOTFALL COUNTER - FRESH START WITH PROPER VIDEO

📹 BEST OPTIONS FOR DOORWAY/CORRIDOR COUNTING:

1️⃣  MALL DATASET (RECOMMENDED)
   - 2000 frames, 60K+ pedestrians
   - Static camera, mall entrance
   - Perfect for entry/exit counting

2️⃣  PETS2009 DATASET
   - Surveillance footage
   - Clear line crossing scenarios
   - Multiple people, realistic

3️⃣  UCSD PEDESTRIAN DATASET
   - Walkway/corridor scenarios
   - Static camera, clear view

4️⃣  FREE STOCK VIDEO (EASIEST)
   - Pexels/Pixabay doorway videos
   - No registration needed
   - Ready to download

💡 RECOMMENDED APPROACH:

We'll now try to automatically download a **perfect sample pedestrian video**:
- Static security camera
- Good crowd movement
- Ideal for entry/exit counting

🔄 Checking / Creating videos folder...

🔄 Downloading sample pedestrian video...

🔄 Trying to download: towncentre.avi ...
❌ Error downloading towncentre.avi: HTTP Error 404: Not Found

🔄 Trying to download: mot17.webm ...
✅ Successfully Downloaded: 

In [2]:
# Step 1: Verify Downloaded Video (LOCAL VERSION)
import cv2
import os

print("="*70)
print("STEP 1: Video Verification")
print("="*70)

video_path = "videos/ddd.mp4"   
# video_path = "videos/mot17.webm"

if not os.path.exists(video_path):
    print(f"❌ File not found at: {video_path}")
    print("➡️ Check the filename and be sure it is placed inside the 'videos' folder.")
else:
    cap = cv2.VideoCapture(video_path)

    if cap.isOpened():
        FRAME_WIDTH = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        FRAME_HEIGHT = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        FPS = int(cap.get(cv2.CAP_PROP_FPS))
        TOTAL_FRAMES = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        DURATION = TOTAL_FRAMES / FPS if FPS > 0 else 0

        print(f"✅ Video loaded successfully!\n")
        print("📹 Video Properties:")
        print(f"   - Resolution: {FRAME_WIDTH} x {FRAME_HEIGHT}")
        print(f"   - FPS: {FPS}")
        print(f"   - Total Frames: {TOTAL_FRAMES}")
        print(f"   - Duration: {DURATION:.2f} seconds")
        print(f"   - File: {video_path}")

        # Display first frame (LOCAL display)
        ret, frame = cap.read()
        if ret:
            print("\n🖼️ First Frame Preview (Press any key to close window)")
            display_frame = cv2.resize(frame, (800, int(800 * FRAME_HEIGHT / FRAME_WIDTH)))
            cv2.imshow("Video Preview", display_frame)
            cv2.waitKey(0) 
            cv2.destroyAllWindows()
        else:
            print("⚠️ Could not read first frame.")

        cap.release()
        print("\n✅ Video ready for processing!")
    else:
        print("❌ Could not open video file (maybe codec issue?)")

print("\n" + "="*70)


STEP 1: Video Verification
✅ Video loaded successfully!

📹 Video Properties:
   - Resolution: 1280 x 720
   - FPS: 25
   - Total Frames: 341
   - Duration: 13.64 seconds
   - File: videos/ddd.mp4

🖼️ First Frame Preview (Press any key to close window)

✅ Video ready for processing!



In [4]:
# =============================================================================
# COMPLETE FOOTFALL COUNTER - ALL-IN-ONE (FIXED)
# Smart Detection + Processing + Heatmap + CSV + Everything
# =============================================================================

print("="*70)
print("🚀 COMPLETE FOOTFALL COUNTER SYSTEM")
print("   Smart Line Detection | Processing | Heatmap | Logs")
print("="*70)

# -----------------------------------------------------------------------------
# IMPORTS & SETUP
# -----------------------------------------------------------------------------
from ultralytics import YOLO
from collections import deque, defaultdict
import cv2
import numpy as np
import time
import pandas as pd
from IPython.display import FileLink
import os

# CREATE OUTPUT DIRECTORY
os.makedirs('outputs', exist_ok=True)
print("\n✅ Output directory ready!")

print("\n📦 Loading Model...")
model = YOLO('yolov8n.pt')
CONFIDENCE_THRESHOLD = 0.30
IOU_THRESHOLD = 0.5
PERSON_CLASS_ID = 0
print("✅ Model loaded!")

# Video path
video_path = "videos/ddd.mp4"
# video_path = "videos/mot17.webm"

# Get video properties
cap = cv2.VideoCapture(video_path)
FRAME_WIDTH = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
FRAME_HEIGHT = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
FPS = int(cap.get(cv2.CAP_PROP_FPS))
TOTAL_FRAMES = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
cap.release()

print(f"\n📹 Video: {FRAME_WIDTH}x{FRAME_HEIGHT}, {TOTAL_FRAMES} frames, {FPS} FPS")

# -----------------------------------------------------------------------------
# SMART LINE DETECTION - Movement Vector Analysis
# -----------------------------------------------------------------------------
print(f"\n🧠 SMART LINE DETECTION")
print("="*70)
print("   Analyzing movement patterns to determine optimal line...\n")

cap = cv2.VideoCapture(video_path)

person_tracks = defaultdict(list)
movement_vectors = []
frame_count = 0
max_analysis_frames = min(200, TOTAL_FRAMES)

while frame_count < max_analysis_frames:
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1

    results = model.track(
        frame,
        conf=CONFIDENCE_THRESHOLD,
        iou=IOU_THRESHOLD,
        classes=[PERSON_CLASS_ID],
        persist=True,
        verbose=False,
        tracker="bytetrack.yaml"
    )

    boxes = results[0].boxes

    if boxes.id is not None:
        track_ids = boxes.id.cpu().numpy().astype(int)
        xyxy = boxes.xyxy.cpu().numpy()

        for track_id, box in zip(track_ids, xyxy):
            x1, y1, x2, y2 = box

            width = x2 - x1
            height = y2 - y1
            aspect_ratio = height / width if width > 0 else 0

            if aspect_ratio < 1.2 or aspect_ratio > 5.0:
                continue

            cx = int((x1 + x2) / 2)
            cy = int((y1 + y2) / 2)

            person_tracks[track_id].append((cx, cy, frame_count))

            if len(person_tracks[track_id]) >= 2:
                prev_pos = person_tracks[track_id][-2]
                curr_pos = person_tracks[track_id][-1]

                dx = curr_pos[0] - prev_pos[0]
                dy = curr_pos[1] - prev_pos[1]

                movement_magnitude = np.sqrt(dx**2 + dy**2)
                if movement_magnitude > 3:
                    movement_vectors.append((dx, dy))

    if frame_count % 50 == 0:
        print(f"   Analyzed: {frame_count}/{max_analysis_frames} frames")

cap.release()

# Analyze movement to determine line type
if len(movement_vectors) < 10:
    print(f"\n⚠️  Insufficient data ({len(movement_vectors)} vectors) - using default VERTICAL")
    LINE_TYPE = "VERTICAL"
    LINE_X = FRAME_WIDTH // 2
    LINE_START = (LINE_X, 0)
    LINE_END = (LINE_X, FRAME_HEIGHT)
else:
    dx_values = [vec[0] for vec in movement_vectors]
    dy_values = [vec[1] for vec in movement_vectors]

    total_horizontal = sum(abs(dx) for dx in dx_values)
    total_vertical = sum(abs(dy) for dy in dy_values)

    avg_dx = np.mean(np.abs(dx_values))
    avg_dy = np.mean(np.abs(dy_values))

    horizontal_ratio = total_horizontal / (total_vertical + 1)

    print(f"\n📊 Movement Analysis:")
    print(f"   Vectors Analyzed: {len(movement_vectors)}")
    print(f"   Horizontal Movement: {total_horizontal:.0f} px")
    print(f"   Vertical Movement: {total_vertical:.0f} px")
    print(f"   Avg Speed (H/V): {avg_dx:.2f} / {avg_dy:.2f} px/frame")
    print(f"   H/V Ratio: {horizontal_ratio:.2f}")

    if horizontal_ratio > 1.3:
        LINE_TYPE = "VERTICAL"
        LINE_X = FRAME_WIDTH // 2
        LINE_START = (LINE_X, 0)
        LINE_END = (LINE_X, FRAME_HEIGHT)
        print(f"\n✅ DECISION: Horizontal movement dominant")
        print(f"   → Using VERTICAL line at X={LINE_X}")
    else:
        LINE_TYPE = "HORIZONTAL"
        LINE_Y = FRAME_HEIGHT // 2
        LINE_START = (0, LINE_Y)
        LINE_END = (FRAME_WIDTH, LINE_Y)
        print(f"\n✅ DECISION: Vertical movement dominant")
        print(f"   → Using HORIZONTAL line at Y={LINE_Y}")

print("\n" + "="*70)

# -----------------------------------------------------------------------------
# FULL VIDEO PROCESSING
# -----------------------------------------------------------------------------
print(f"\n🎬 VIDEO PROCESSING")
print("="*70)

track_history = {}
track_last_direction = {}
entry_count = 0
exit_count = 0
crossing_log = []
heatmap_accumulator = np.zeros((FRAME_HEIGHT, FRAME_WIDTH), dtype=np.float32)

TRAJECTORY_LENGTH = 10
CROSSING_THRESHOLD = 30

output_path = "outputs/footfall_complete.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, FPS, (FRAME_WIDTH, FRAME_HEIGHT))

cap = cv2.VideoCapture(video_path)
frame_count = 0
start_time = time.time()

print("\nProcessing...\n")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1

    results = model.track(
        frame,
        conf=CONFIDENCE_THRESHOLD,
        iou=IOU_THRESHOLD,
        classes=[PERSON_CLASS_ID],
        persist=True,
        verbose=False,
        tracker="bytetrack.yaml"
    )

    boxes = results[0].boxes

    if boxes.id is not None:
        track_ids = boxes.id.cpu().numpy().astype(int)
        xyxy = boxes.xyxy.cpu().numpy()

        for track_id, box in zip(track_ids, xyxy):
            x1, y1, x2, y2 = map(int, box)

            width = x2 - x1
            height = y2 - y1
            aspect_ratio = height / width if width > 0 else 0

            if aspect_ratio < 1.2 or aspect_ratio > 5.0:
                continue

            if width > FRAME_WIDTH * 0.4 or height > FRAME_HEIGHT * 0.6:
                continue

            centroid_x = (x1 + x2) // 2
            centroid_y = (y1 + y2) // 2
            centroid = (centroid_x, centroid_y)

            # ACCURATE HEATMAP: Accumulate every detection
            cv2.circle(heatmap_accumulator, centroid, 15, 1, -1)

            if track_id not in track_history:
                track_history[track_id] = deque(maxlen=TRAJECTORY_LENGTH)
                track_last_direction[track_id] = None

            if len(track_history[track_id]) > 0:
                if LINE_TYPE == "VERTICAL":
                    prev_x = track_history[track_id][-1][0]
                    curr_x = centroid_x

                    if prev_x < LINE_X and curr_x >= LINE_X:
                        if abs(curr_x - LINE_X) < CROSSING_THRESHOLD:
                            if track_last_direction[track_id] != 'entry':
                                entry_count += 1
                                track_last_direction[track_id] = 'entry'
                                crossing_log.append({
                                    'frame': frame_count,
                                    'id': track_id,
                                    'action': 'ENTRY',
                                    'time': round(frame_count / FPS, 2),
                                    'position_x': centroid_x,
                                    'position_y': centroid_y
                                })
                                print(f"   🟢 ENTRY | ID:{track_id} | Frame:{frame_count} | Total:{entry_count}")

                    elif prev_x > LINE_X and curr_x <= LINE_X:
                        if abs(curr_x - LINE_X) < CROSSING_THRESHOLD:
                            if track_last_direction[track_id] != 'exit':
                                exit_count += 1
                                track_last_direction[track_id] = 'exit'
                                crossing_log.append({
                                    'frame': frame_count,
                                    'id': track_id,
                                    'action': 'EXIT',
                                    'time': round(frame_count / FPS, 2),
                                    'position_x': centroid_x,
                                    'position_y': centroid_y
                                })
                                print(f"   🔴 EXIT | ID:{track_id} | Frame:{frame_count} | Total:{exit_count}")

                else:  # HORIZONTAL
                    prev_y = track_history[track_id][-1][1]
                    curr_y = centroid_y

                    if prev_y < LINE_Y and curr_y >= LINE_Y:
                        if abs(curr_y - LINE_Y) < CROSSING_THRESHOLD:
                            if track_last_direction[track_id] != 'entry':
                                entry_count += 1
                                track_last_direction[track_id] = 'entry'
                                crossing_log.append({
                                    'frame': frame_count,
                                    'id': track_id,
                                    'action': 'ENTRY',
                                    'time': round(frame_count / FPS, 2),
                                    'position_x': centroid_x,
                                    'position_y': centroid_y
                                })
                                print(f"   🟢 ENTRY | ID:{track_id} | Frame:{frame_count} | Total:{entry_count}")

                    elif prev_y > LINE_Y and curr_y <= LINE_Y:
                        if abs(curr_y - LINE_Y) < CROSSING_THRESHOLD:
                            if track_last_direction[track_id] != 'exit':
                                exit_count += 1
                                track_last_direction[track_id] = 'exit'
                                crossing_log.append({
                                    'frame': frame_count,
                                    'id': track_id,
                                    'action': 'EXIT',
                                    'time': round(frame_count / FPS, 2),
                                    'position_x': centroid_x,
                                    'position_y': centroid_y
                                })
                                print(f"   🔴 EXIT | ID:{track_id} | Frame:{frame_count} | Total:{exit_count}")

            track_history[track_id].append(centroid)

            if track_last_direction.get(track_id) == 'exit':
                color = (0, 0, 255)
            elif track_last_direction.get(track_id) == 'entry':
                color = (0, 255, 0)
            else:
                color = (255, 165, 0)

            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

            label = f"ID:{track_id}"
            (text_w, text_h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
            cv2.rectangle(frame, (x1, y1 - text_h - 10), (x1 + text_w + 5, y1), color, -1)
            cv2.putText(frame, label, (x1 + 2, y1 - 5),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    # Draw line
    cv2.line(frame, LINE_START, LINE_END, (0, 255, 255), 5)

    # Labels
    if LINE_TYPE == "VERTICAL":
        cv2.putText(frame, "ENTRY ->", (LINE_X - 140, 50),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        cv2.putText(frame, "<- EXIT", (LINE_X + 20, 50),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
    else:
        cv2.putText(frame, "ENTRY v", (FRAME_WIDTH // 2 - 80, LINE_Y - 10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        cv2.putText(frame, "^ EXIT", (FRAME_WIDTH // 2 - 60, LINE_Y + 35),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

    # Info panel
    cv2.rectangle(frame, (10, 10), (380, 140), (0, 0, 0), -1)
    cv2.rectangle(frame, (10, 10), (380, 140), (255, 255, 255), 3)

    cv2.putText(frame, f"ENTRIES: {entry_count}", (25, 50),
               cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"EXITS: {exit_count}", (25, 90),
               cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.putText(frame, f"NET: {entry_count - exit_count}", (25, 130),
               cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)

    cv2.putText(frame, f"Frame: {frame_count}/{TOTAL_FRAMES}", (FRAME_WIDTH - 250, 30),
               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    out.write(frame)

    if frame_count % 100 == 0:
        elapsed = time.time() - start_time
        print(f"   Progress: {(frame_count/TOTAL_FRAMES)*100:.1f}% | FPS: {frame_count/elapsed:.1f}")

cap.release()
out.release()
total_time = time.time() - start_time

# -----------------------------------------------------------------------------
# GENERATE ACCURATE HEATMAP
# -----------------------------------------------------------------------------
print(f"\n🔥 GENERATING HEATMAP")
print("="*70)

# Apply moderate Gaussian blur for smooth visualization
heatmap_blurred = cv2.GaussianBlur(heatmap_accumulator, (41, 41), 0)

# Normalize to 0-255 range
heatmap_normalized = cv2.normalize(heatmap_blurred, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

# Apply JET colormap (red=hot, blue=cold)
heatmap_colored = cv2.applyColorMap(heatmap_normalized, cv2.COLORMAP_JET)

# Get background frame
cap = cv2.VideoCapture(video_path)
ret, background = cap.read()
cap.release()

# Overlay heatmap on background (50/50 blend)
heatmap_overlay = cv2.addWeighted(background, 0.5, heatmap_colored, 0.5, 0)

# Draw counting line on heatmap
if LINE_TYPE == "VERTICAL":
    cv2.line(heatmap_overlay, LINE_START, LINE_END, (255, 255, 255), 6)
else:
    cv2.line(heatmap_overlay, LINE_START, LINE_END, (255, 255, 255), 6)

# Add title and stats
cv2.rectangle(heatmap_overlay, (10, 10), (FRAME_WIDTH - 10, 120), (0, 0, 0), -1)
cv2.rectangle(heatmap_overlay, (10, 10), (FRAME_WIDTH - 10, 120), (255, 255, 255), 3)

cv2.putText(heatmap_overlay, "MOVEMENT HEATMAP", (20, 50),
           cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 3)
cv2.putText(heatmap_overlay, f"Entries: {entry_count} | Exits: {exit_count} | Net: {entry_count-exit_count}",
           (20, 95), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

# Add legend
legend_x = FRAME_WIDTH - 150
legend_y = FRAME_HEIGHT - 100
cv2.putText(heatmap_overlay, "Activity:", (legend_x, legend_y),
           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
cv2.putText(heatmap_overlay, "HIGH", (legend_x + 10, legend_y + 25),
           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
cv2.putText(heatmap_overlay, "LOW", (legend_x + 10, legend_y + 50),
           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

heatmap_path = "outputs/heatmap_accurate.png"
cv2.imwrite(heatmap_path, heatmap_overlay)

print(f"✅ Heatmap generated: {heatmap_path}")

# -----------------------------------------------------------------------------
# EXPORT CSV LOG
# -----------------------------------------------------------------------------
print(f"\n📄 EXPORTING LOGS")
print("="*70)

if len(crossing_log) > 0:
    df = pd.DataFrame(crossing_log)
    csv_path = "outputs/crossing_log.csv"
    df.to_csv(csv_path, index=False)
    print(f"✅ CSV saved: {csv_path}")
    print(f"   Total crossings logged: {len(df)}")
    print(f"\n📋 First 10 crossings:")
    print(df.head(10).to_string(index=False))
else:
    print("⚠️  No crossings detected")
    csv_path = None

# -----------------------------------------------------------------------------
# FINAL SUMMARY
# -----------------------------------------------------------------------------
print(f"\n" + "="*70)
print("🎉 COMPLETE SYSTEM FINISHED!")
print("="*70)
print(f"""
🧠 SMART DETECTION:
   Line Type: {LINE_TYPE} (auto-detected)
   Detection Method: Movement vector analysis

📊 FINAL COUNTS:
   🟢 Entries: {entry_count}
   🔴 Exits: {exit_count}
   📈 Net Count: {entry_count - exit_count}
   👥 Unique People: {len(track_history)}

⚙️  PERFORMANCE:
   Processing Time: {total_time:.2f}s
   Average FPS: {TOTAL_FRAMES/total_time:.2f}

📦 DELIVERABLES CREATED:
   ✅ Video: {output_path}
   ✅ Heatmap: {heatmap_path}
   ✅ CSV Log: {csv_path if csv_path else 'None'}
""")
print("="*70)

# -----------------------------------------------------------------------------
# DISPLAY RESULTS
# -----------------------------------------------------------------------------
print(f"\n📺 PREVIEW")
print("="*70)

# Display heatmap
print(f"\n🖼️  Movement Heatmap:")
cv2_imshow(cv2.resize(heatmap_overlay, (800, int(800 * FRAME_HEIGHT / FRAME_WIDTH))))

# Display sample frames
print(f"\n🎬 Sample Video Frames:\n")
cap = cv2.VideoCapture(output_path)

for frame_num in [100, 300, 500]:
    if frame_num < TOTAL_FRAMES:
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
        ret, frame = cap.read()
        if ret:
            print(f"Frame {frame_num}:")
            cv2_imshow(cv2.resize(frame, (800, int(800 * FRAME_HEIGHT / FRAME_WIDTH))))
            print()

cap.release()

# -----------------------------------------------------------------------------
# DOWNLOAD LINKS
# -----------------------------------------------------------------------------
print("="*70)
print("💾 DOWNLOAD FILES")
print("="*70)
print("\n📥 Click links below to download:\n")

display(FileLink(output_path))
display(FileLink(heatmap_path))
if csv_path:
    display(FileLink(csv_path))

print("\n✅ ALL DONE! Your footfall counter system is complete.")
print("="*70)

🚀 COMPLETE FOOTFALL COUNTER SYSTEM
   Smart Line Detection | Processing | Heatmap | Logs

✅ Output directory ready!

📦 Loading Model...
✅ Model loaded!

📹 Video: 1280x720, 341 frames, 25 FPS

🧠 SMART LINE DETECTION
   Analyzing movement patterns to determine optimal line...

   Analyzed: 50/200 frames
   Analyzed: 100/200 frames
   Analyzed: 150/200 frames
   Analyzed: 200/200 frames

📊 Movement Analysis:
   Vectors Analyzed: 1080
   Horizontal Movement: 1726 px
   Vertical Movement: 4558 px
   Avg Speed (H/V): 1.60 / 4.22 px/frame
   H/V Ratio: 0.38

✅ DECISION: Vertical movement dominant
   → Using HORIZONTAL line at Y=360


🎬 VIDEO PROCESSING

Processing...

   🟢 ENTRY | ID:88 | Frame:8 | Total:1
   🟢 ENTRY | ID:84 | Frame:42 | Total:2
   🔴 EXIT | ID:85 | Frame:44 | Total:1
   🔴 EXIT | ID:121 | Frame:64 | Total:2
   🟢 ENTRY | ID:98 | Frame:92 | Total:3
   Progress: 29.3% | FPS: 42.2
   🔴 EXIT | ID:86 | Frame:110 | Total:3
   🟢 ENTRY | ID:100 | Frame:132 | Total:4
   🟢 ENTRY | ID:118

/home/sheesh/Desktop/yolo/outputs/footfall_complete.mp4

/home/sheesh/Desktop/yolo/outputs/heatmap_accurate.png

/home/sheesh/Desktop/yolo/outputs/crossing_log.csv


✅ ALL DONE! Your footfall counter system is complete.


# Real-Tme Analysis


In [17]:
#!/usr/bin/env python3
"""
Real-Time Footfall Counter - Simple Logic
Track IDs that have crossed, don't count again
"""

from ultralytics import YOLO
import cv2
import time
import os

# =============================================================================
# CONFIGURATION
# =============================================================================
CONFIDENCE = 0.3
PERSON_CLASS = 0

os.makedirs('outputs', exist_ok=True)

print("="*70)
print("REAL-TIME FOOTFALL COUNTER")
print("="*70)
print("\nControls: q=quit | r=reset | s=screenshot\n")

# =============================================================================
# LOAD MODEL
# =============================================================================
print("Loading model...")
model = YOLO('yolov8n.pt')
print("✓ Model loaded\n")

# =============================================================================
# INITIALIZE CAMERA
# =============================================================================
print("Opening camera...")
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("ERROR: Cannot open camera!")
    exit(1)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

WIDTH = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
HEIGHT = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

print(f"✓ Camera ready: {WIDTH}x{HEIGHT}\n")

# =============================================================================
# DETERMINE LINE ORIENTATION
# =============================================================================
print("Analyzing movement (5 seconds)...")

movements_x = []
movements_y = []
temp_positions = {}

analysis_start = time.time()
while time.time() - analysis_start < 5:
    ret, frame = cap.read()
    if not ret:
        continue
    
    results = model.track(frame, conf=CONFIDENCE, classes=[PERSON_CLASS], 
                         persist=True, verbose=False, tracker="bytetrack.yaml")
    
    if results[0].boxes.id is not None:
        ids = results[0].boxes.id.cpu().numpy().astype(int)
        boxes = results[0].boxes.xyxy.cpu().numpy()
        
        for tid, box in zip(ids, boxes):
            cx = int((box[0] + box[2]) / 2)
            cy = int((box[1] + box[3]) / 2)
            
            if tid in temp_positions:
                prev_cx, prev_cy = temp_positions[tid]
                movements_x.append(abs(cx - prev_cx))
                movements_y.append(abs(cy - prev_cy))
            
            temp_positions[tid] = (cx, cy)
    
    cv2.putText(frame, "ANALYZING...", (50, 50), 
               cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 255), 3)
    cv2.imshow('Setup', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        exit(0)

cv2.destroyAllWindows()

total_x = sum(movements_x) if movements_x else 0
total_y = sum(movements_y) if movements_y else 0

if total_x > total_y * 1.2:
    LINE_TYPE = "VERTICAL"
    LINE_POS = WIDTH // 2
    print(f"✓ Horizontal movement → VERTICAL line at x={LINE_POS}\n")
else:
    LINE_TYPE = "HORIZONTAL"
    LINE_POS = HEIGHT // 2
    print(f"✓ Vertical movement → HORIZONTAL line at y={LINE_POS}\n")

# =============================================================================
# TRACKING STATE - SIMPLE LOGIC
# =============================================================================
# Store previous positions: {track_id: (x, y)}
prev_positions = {}

# Store which IDs have already been counted for each direction
counted_entry_ids = set()  # IDs that have entered
counted_exit_ids = set()   # IDs that have exited

entry_count = 0
exit_count = 0

# FPS tracking
fps_time = time.time()
fps_count = 0
fps = 0

print("="*70)
print("SYSTEM ACTIVE")
print("="*70)

# =============================================================================
# MAIN LOOP
# =============================================================================
try:
    while True:
        ret, frame = cap.read()
        if not ret:
            continue
        
        # Run detection & tracking
        results = model.track(
            frame,
            conf=CONFIDENCE,
            classes=[PERSON_CLASS],
            persist=True,
            verbose=False,
            tracker="bytetrack.yaml"
        )
        
        current_ids = set()
        
        # Process detections
        if results[0].boxes.id is not None:
            track_ids = results[0].boxes.id.cpu().numpy().astype(int)
            boxes = results[0].boxes.xyxy.cpu().numpy()
            
            for tid, box in zip(track_ids, boxes):
                current_ids.add(tid)
                
                x1, y1, x2, y2 = map(int, box)
                cx = (x1 + x2) // 2
                cy = (y1 + y2) // 2
                
                # Check for line crossing
                if tid in prev_positions:
                    prev_cx, prev_cy = prev_positions[tid]
                    
                    if LINE_TYPE == "VERTICAL":
                        # Check if crossed vertical line
                        if prev_cx < LINE_POS and cx >= LINE_POS:
                            # Crossed left to right = ENTRY
                            if tid not in counted_entry_ids:
                                entry_count += 1
                                counted_entry_ids.add(tid)
                                # Remove from exit set (can cross back)
                                counted_exit_ids.discard(tid)
                                print(f"[ENTRY] ID {tid} → Entries: {entry_count}")
                        
                        elif prev_cx > LINE_POS and cx <= LINE_POS:
                            # Crossed right to left = EXIT
                            if tid not in counted_exit_ids:
                                exit_count += 1
                                counted_exit_ids.add(tid)
                                # Remove from entry set (can cross back)
                                counted_entry_ids.discard(tid)
                                print(f"[EXIT] ID {tid} → Exits: {exit_count}")
                    
                    else:  # HORIZONTAL
                        # Check if crossed horizontal line
                        if prev_cy < LINE_POS and cy >= LINE_POS:
                            # Crossed top to bottom = ENTRY
                            if tid not in counted_entry_ids:
                                entry_count += 1
                                counted_entry_ids.add(tid)
                                counted_exit_ids.discard(tid)
                                print(f"[ENTRY] ID {tid} → Entries: {entry_count}")
                        
                        elif prev_cy > LINE_POS and cy <= LINE_POS:
                            # Crossed bottom to top = EXIT
                            if tid not in counted_exit_ids:
                                exit_count += 1
                                counted_exit_ids.add(tid)
                                counted_entry_ids.discard(tid)
                                print(f"[EXIT] ID {tid} → Exits: {exit_count}")
                
                # Update position for next frame
                prev_positions[tid] = (cx, cy)
                
                # Determine color based on counting status
                if tid in counted_entry_ids:
                    color = (0, 255, 0)  # Green - entered
                elif tid in counted_exit_ids:
                    color = (0, 0, 255)  # Red - exited
                else:
                    color = (255, 165, 0)  # Orange - not counted yet
                
                # Draw bounding box
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 3)
                
                # Draw ID label
                label = f"ID:{tid}"
                cv2.rectangle(frame, (x1, y1-30), (x1+100, y1), color, -1)
                cv2.putText(frame, label, (x1+5, y1-8),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        
        # Clean up positions for IDs no longer detected
        prev_positions = {tid: pos for tid, pos in prev_positions.items() if tid in current_ids}
        
        # Draw counting line
        if LINE_TYPE == "VERTICAL":
            cv2.line(frame, (LINE_POS, 0), (LINE_POS, HEIGHT), (0, 255, 255), 5)
            
            y_mid = HEIGHT // 2
            cv2.arrowedLine(frame, (LINE_POS - 120, y_mid), (LINE_POS - 30, y_mid),
                           (0, 255, 0), 4, tipLength=0.4)
            cv2.putText(frame, "ENTRY", (LINE_POS - 190, y_mid - 15),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
            
            cv2.arrowedLine(frame, (LINE_POS + 120, y_mid), (LINE_POS + 30, y_mid),
                           (0, 0, 255), 4, tipLength=0.4)
            cv2.putText(frame, "EXIT", (LINE_POS + 130, y_mid - 15),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
        else:
            cv2.line(frame, (0, LINE_POS), (WIDTH, LINE_POS), (0, 255, 255), 5)
            
            x_mid = WIDTH // 2
            cv2.arrowedLine(frame, (x_mid, LINE_POS - 120), (x_mid, LINE_POS - 30),
                           (0, 255, 0), 4, tipLength=0.4)
            cv2.putText(frame, "ENTRY", (x_mid - 60, LINE_POS - 130),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
            
            cv2.arrowedLine(frame, (x_mid, LINE_POS + 120), (x_mid, LINE_POS + 30),
                           (0, 0, 255), 4, tipLength=0.4)
            cv2.putText(frame, "EXIT", (x_mid - 50, LINE_POS + 140),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
        
        # Info panel
        cv2.rectangle(frame, (10, 10), (350, 170), (0, 0, 0), -1)
        cv2.rectangle(frame, (10, 10), (350, 170), (255, 255, 255), 3)
        
        cv2.putText(frame, f"ENTRIES: {entry_count}", (25, 50),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f"EXITS: {exit_count}", (25, 95),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.putText(frame, f"NET: {entry_count - exit_count}", (25, 140),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 0), 2)
        
        # FPS
        fps_count += 1
        if time.time() - fps_time >= 1:
            fps = fps_count
            fps_count = 0
            fps_time = time.time()
        
        cv2.putText(frame, f"FPS: {fps}", (WIDTH - 150, 40),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        
        # Show frame
        cv2.imshow('Real-Time Footfall Counter', frame)
        
        # Key controls
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('r'):
            entry_count = 0
            exit_count = 0
            counted_entry_ids.clear()
            counted_exit_ids.clear()
            prev_positions.clear()
            print("\n[RESET]\n")
        elif key == ord('s'):
            path = f"outputs/screenshot_{int(time.time())}.jpg"
            cv2.imwrite(path, frame)
            print(f"\n[SAVED: {path}]\n")

except KeyboardInterrupt:
    print("\n\nInterrupted")

finally:
    cap.release()
    cv2.destroyAllWindows()
    
    print("\n" + "="*70)
    print("SESSION COMPLETE")
    print("="*70)
    print(f"Entries: {entry_count}")
    print(f"Exits: {exit_count}")
    print(f"Net: {entry_count - exit_count}")
    print("="*70 + "\n")

REAL-TIME FOOTFALL COUNTER

Controls: q=quit | r=reset | s=screenshot

Loading model...
✓ Model loaded

Opening camera...
✓ Camera ready: 640x480

Analyzing movement (5 seconds)...
✓ Horizontal movement → VERTICAL line at x=320

SYSTEM ACTIVE
[EXIT] ID 1 → Exits: 1
[EXIT] ID 6 → Exits: 2
[ENTRY] ID 15 → Entries: 1

SESSION COMPLETE
Entries: 1
Exits: 2
Net: -1

